In [ ]:
from pathlib import Path
import json
import pandas as pd
import pickle as pkl

In [ ]:
BASE_PATH = Path('/home/detectron2/datasets/nia/')
IMG_PATH = BASE_PATH / '1.원천데이터'
ANNO_PATH = BASE_PATH / '2.라벨링데이터'

In [ ]:
# 가시광 데이터 필터링
def is_visible_data(item):
    cond = item.match('image_B/*.png*') or item.match('image_F/*.png*') or item.match('image_L/*.png*') or item.match('image_R/*.png*')
    return cond

In [ ]:
img_paths = list(IMG_PATH.rglob('*.png'))
img_names= [item.name for item in img_paths]

anno_paths = list(ANNO_PATH.rglob('*.json'))
anno_names= [item.name for item in anno_paths]

visible_anno_paths = list(filter(is_visible_data, anno_paths))
visible_anno_paths = list(filter(lambda x: '._' not in x.as_posix(), visible_anno_paths))
visible_anno_names = [item.name for item in visible_anno_paths]

visible_img_paths = list(filter(is_visible_data, img_paths))
visible_img_names = [item.name for item in visible_img_paths]

visible_anno_names_wo_json = [item.rstrip('.json') for item in visible_anno_names]

df_visible_img = pd.DataFrame({'filename': visible_img_names, 'imgpath': visible_img_paths}).set_index('filename')
df_visible_anno = pd.DataFrame({'filename': visible_anno_names_wo_json, 'annopath': visible_anno_paths}).set_index('filename')

df_visible = pd.concat([df_visible_img, df_visible_anno], axis=1).dropna(how='any')

In [ ]:
# visible 데이터 오류 검출
# 가시광 데이터 annotations에서 segmentation이 None으로 표기된 경우들 + len(annotations)==0인 경우들 검출하기
# 가시광 데이터 annotations에서 segmentation['coord']['points']가 list가 아니거나 list인데 길이가 0인 경우 검출하기
# json 파일에서 필요한 정보만 추출: 'images', 'annotations'
img_wo_anno = list()
img_wo_seg = list()
img_wo_points = list()

for item in df_visible.annopath:
    error_flag = False
    error_flag1 = False
    with item.open() as f:
        item_json = json.load(f)
    
    if len(item_json['annotations']) == 0:
        img_wo_anno.append(item.name.rstrip('.json'))
    else:
        for anno in item_json['annotations']:
            if anno['segmentation'] is None:
                error_flag = True
                import ipdb;ipdb.set_trace()
                break

            if type(anno['segmentation']['coord']['points']) is not list :
                error_flag1 = True
                break
            
            else:
                if len(anno['segmentation']['coord']['points']) == 0:
                    error_flag1 = True
                    break
        
        if error_flag == True:
            img_wo_seg.append(item.name.rstrip('.json'))
        if error_flag1 == True:
            img_wo_points.append(item.name.rstrip('.json'))

In [ ]:
img_w_error = img_wo_anno + img_wo_seg + img_wo_points
with open('/home/detectron2/nia/img_w_bug.pkl', 'wb') as f:
    pkl.dump(img_w_error, f)